Library

In [1]:
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Path ke dataset
dataset_path = '/content/drive/MyDrive/Colab Notebooks/dataSIBI'

Tuning

In [ ]:
# Fungsi untuk mengekstrak frame dari video dan menyimpannya sebagai gambar
def extract_frames(video_path, output_folder, max_frames=30):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    frame_list = []

    while cap.isOpened() and frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        # Resize frame agar sesuai dengan input model
        frame = cv2.resize(frame, (224, 224))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Ubah dari BGR ke RGB
        frame_list.append(frame)
        frame_count += 1

    cap.release()
    return frame_list

# Proses setiap folder di dalam dataset
frames = []
labels = []
label_map = {}  # Menyimpan mapping label ke indeks

label_index = 0
for folder in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder)
    if os.path.isdir(folder_path):
        # Tambahkan label ke mapping jika belum ada
        if folder not in label_map:
            label_map[folder] = label_index
            label_index += 1

        # Proses video dalam folder
        for video_file in os.listdir(folder_path):
            if video_file.endswith('.webm'):
                video_path = os.path.join(folder_path, video_file)
                video_frames = extract_frames(video_path, folder_path)
                frames.extend(video_frames)
                labels.extend([label_map[folder]] * len(video_frames))

# Konversi frames dan labels ke array numpy
frames = np.array(frames)
labels = np.array(labels)

# Normalisasi data
frames = frames / 255.0  # Normalisasi nilai pixel ke [0, 1]

# Membagi data menjadi training set
train_dataset = tf.data.Dataset.from_tensor_slices((frames, labels))
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)

# Buat model menggunakan pre-trained EfficientNetB0
base_model = EfficientNetB0(weights='imagenet', input_shape=(224, 224, 3), include_top=False)
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(label_map), activation='softmax')  # Jumlah kelas
])

# Kompilasi model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Training

In [ ]:
# Melatih model
history = model.fit(
    train_dataset,
    epochs=10  # Sesuaikan jumlah epoch sesuai kebutuhan
)


In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/trained_model.h5')

Implementasi

In [ ]:
# Load model TensorFlow (sudah dilatih sebelumnya)
model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/trained_model.h5')  # Path yang benar

# Fungsi untuk memproses frame dan membuat prediksi
def predict_hand_gesture(frame):
    frame_resized = cv2.resize(frame, (224, 224))
    frame_normalized = frame_resized / 255.0
    frame_expanded = np.expand_dims(frame_normalized, axis=0)
    prediction = model.predict(frame_expanded)
    class_index = np.argmax(prediction)
    return class_index, prediction[0][class_index]

# Inisialisasi webcam
cap = cv2.VideoCapture(0)
recording = False

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    if recording:
        class_index, confidence = predict_hand_gesture(frame_rgb)
        label = f"{class_labels[class_index]}: {confidence:.2f}"
        cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow('Hand Gesture Recognition', frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('s'):  # Tekan 's' untuk mulai merekam
        recording = True
        print("Recording started...")

    elif key == ord('q'):  # Tekan 'q' untuk keluar
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
